# Six-bar Mechanism Balancing

In [1]:
from BetaShF import ShF
from BetaShM import ShM 
import numpy as np 
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
from cnsg_differential_evolution import cnsg_differential_evolution
import time

In [2]:
a = 0.5

### Utils

In [3]:
def cleanData(samples, fitness, forces, moments):
    filterF = forces < 1
    filterM = moments < 1
    f = np.logical_and(filterF, filterM)
    print(f.shape)
    return samples[f], fitness[f], forces[f], moments[f]

In [14]:
def logSample(now, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open("./experiments/"+name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(str(now) + "_Population.txt", s)
    appendToFile(str(now) + "_Fitness.txt", str(fitness))
    appendToFile(str(now) + "_ShForces.txt", str(force))
    appendToFile(str(now) + "_ShMoments.txt", str(moment))

### Problem Definition

##### Contraints

$$-0.16m <= x_{cn},y_{cn} <= 0.16m$$

$$0.005m <= t_{cn} <= 0.04m$$


##### Objective Function

In [5]:
def objective_function(s, ShF, ShM, a): #c is a constant that distributes the weight among the functions.
    return a*ShF(s) + (1-a)*ShM(s)

### Define boundaries

In [6]:
# Bounds for each variable
nVar = 5
bounds = []
for i in range(1,nVar*3+1):
    if(i%3==0): bounds.append([0.005,0.04])
    else: bounds.append([-0.16, 0.16])
bounds = np.array(bounds)
print('bounds',bounds)

bounds [[-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]]


## Gradient Descent

### CG

In [7]:
def gradiente_conjugado(X0,f,MaxIter=100,eps=1e-5):
    k=0
    X = X0
    G = Vf(X,ShF, ShM, f)
    normaGradiente = np.linalg.norm(G)
    P = -G
    curr_fit = f(X,ShF, ShM)
    while(k<=MaxIter and normaGradiente>=eps):
        Ap = V2fTd(X,ShF, ShM,P,f)
        alpha = np.dot(-P,G) / np.dot(P,Ap)
        X = X + alpha*P
        G_ = np.copy(G)
        G = G + alpha*Ap
        normaGradiente = np.linalg.norm(G)
        B = -np.dot(G,G)/np.dot(G_,G_)
        P = -G + B*P
        k = k+1
        print("#",k, ", fit: ", f(X,ShF, ShM))
        if f(X,ShF, ShM) <= curr_fit:
            best_x = X
            curr_fit = f(best_x,ShF, ShM)
    return best_x

### GD

In [8]:
import numpy as np
#################### Descenso de Gradiente con diferenciación finita
eps = 1e-5
def Gradiente(X,f, ShF, ShM):
    n = len(X)
    G = np.zeros((n),float)
    incX = np.zeros((n),float)
    for i in range(n):
        incX[i] = eps
        G[i] = (f(X+incX, ShF, ShM, a)-f(X, ShF, ShM, a))/eps
        incX[i] = 0
    return G
def getStepSize(a,m,X,P,G,f):
    c0 = 1e-4
    c1 = 2
    c2 = 5
    c3 = 3
    eps = 1e-8
    alpha = a
    while f(X+alpha*P, ShF, ShM, a) > f(X, ShF, ShM, a)+c0*np.dot(G,P):
        m = 0
        alpha = alpha/c1
        if alpha<=eps:
            break
    m += 1
    if m>=c2:
        m=0
        alpha = c3*alpha
    return alpha,m

def Gradient_Descent(X0,f,bounds,MaxIter=1000,alpha=1e-3):
    k=0
    X = X0
    G = Gradiente(X,f, ShF, ShM)
    normaGradiente = np.linalg.norm(G)
    m = 0
    while(k<=MaxIter and normaGradiente>=eps):
        G = Gradiente(X,f, ShF, ShM)
        normaGradiente = np.linalg.norm(G)
        P = - G / normaGradiente
        alpha,m = getStepSize(alpha,m,X,P,G,f)
        X = X + alpha*P
        X = np.clip(X,bounds[:,0],bounds[:,1])
        # verficar bounds: si x() no coincide con sus bounds correspondientes
        k = k+1
        if k%100 == 0:
            print("\t\t#",k,f(X, ShF, ShM, a))
    return X


In [9]:
s=bounds[:,0]
r = objective_function(s, ShF, ShM,0.5)
print('OF:',r)

OF: 46.604883632565645


In [10]:
def random_start(bounds):
    # print(bounds.shape[0])
    arr = np.zeros(bounds.shape[0])
    for i, tupl in enumerate(bounds):
        rand = np.random.uniform(tupl[0], tupl[1])
        # print(i, rand)
        arr[i] = rand 
    return arr
    
print(random_start(bounds))

[-0.01271237 -0.07355877  0.01803034 -0.12579818  0.09111245  0.03189632
 -0.1226464  -0.05342432  0.01212005  0.11049056 -0.12049435  0.01946655
 -0.08877147  0.14706508  0.03684394]


### GD

In [15]:
# it = 80
# n = 4
def GD(GD_alph, iter, of_alpha):
    startTime_GD = str(int(time.time()))
    import numpy as np  
    n = 100
    sols = np.zeros((n, 2))
    best, bestSol = 10, None
    eTime = 0

    print(f'\n* Number of iterations: {iter}')
    for i in range(n):
        r = random_start(bounds)
        fitness = objective_function(r, ShF, ShM, of_alpha)
        print(f"\t\nInitial Fitness: {fitness} in #{i}")
        start = time.perf_counter()
        r = Gradient_Descent(r,objective_function, bounds,MaxIter=iter, alpha=GD_alph)
        end = time.perf_counter()
        fit_GD = objective_function(r,ShF,ShM, of_alpha)
        print("\t  - before: ", fitness)
        print("\t  - after (GD): ", fit_GD)
        if fit_GD < best:
            best = fit_GD
            bestSol = r
        eTime += (end-start) #Time in seconds
        logSample(startTime_GD, r, fit_GD, ShF(r), ShM(r)) # (now, sample, fitness, force, moment):
    if n: eTime /= n
    print("Average time of execution:", eTime,"seconds. It was run", n, "times.")
    # return fit_GD

it = 200
of_alpha = np.random.normal(0.5, 0.20)
print(GD(1e-3, it, of_alpha))
print(GD(1e-5, it, of_alpha))
print(GD(1e-8, it, of_alpha))


* Number of iterations: 200
	
Initial Fitness: 37.64609699365422 in #0
		# 100 0.6900564137092842
		# 200 0.7067419325300008
	  - before:  37.64609699365422
	  - after (GD):  0.774924963042817
	
Initial Fitness: 10.432670453165162 in #1
		# 100 0.6654763323688765
		# 200 0.6654763323688765
	  - before:  10.432670453165162
	  - after (GD):  0.6847997782220447
	
Initial Fitness: 22.856012923835923 in #2
		# 100 0.6847271512064855
		# 200 0.6847271209615317
	  - before:  22.856012923835923
	  - after (GD):  0.736183463958648
	
Initial Fitness: 73.05860070542477 in #3
		# 100 3.942794005401961
		# 200 0.71394101470232
	  - before:  73.05860070542477
	  - after (GD):  0.7865321006978068
	
Initial Fitness: 28.825275232117605 in #4
		# 100 0.6451857253807242
		# 200 0.6451857251121679
	  - before:  28.825275232117605
	  - after (GD):  0.6951575009971677
	
Initial Fitness: 41.41858052517796 in #5
		# 100 2.072602025363128
		# 200 0.9097437155665524
	  - before:  41.41858052517796
	  - after (